# SECTIE 1: Setup en Configuratie
# ===============================
# Import benodigde bibliotheken en stel visualisatie-instellingen in


In [1]:

import sys
import os
import json
import traceback
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import MetaTrader5 as mt5
from IPython.display import display
import warnings

# Onderdruk waarschuwingen
warnings.filterwarnings('ignore')

# Visualisatie-instellingen
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (16, 9)
plt.rcParams['lines.linewidth'] = 1.5
sns.set_style("whitegrid")
plt.rcParams['font.size'] = 12
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12

# Project paden configureren
notebook_dir = os.path.abspath('')
project_path = os.path.abspath(os.path.join(notebook_dir, '..'))
print(f"Project pad: {project_path}")
sys.path.append(project_path)

# Hulpfunctie voor foutafhandeling
def display_error(e):
    """Gedetailleerde foutinformatie weergeven"""
    print(f"\n❌ FOUT: {type(e).__name__}: {e}")
    traceback.print_exc()
    print("\nControleer de foutdetails hierboven en los het probleem op voordat u doorgaat.")

# Importeren van TurtleTrader modules
try:
    # Importeer core modules
    from modules.mt5_connector import MT5Connector
    from modules.strategies.turtle_strategy import TurtleStrategy
    from modules.risk_manager import RiskManager
    from modules.backtester import Backtester
    from utils.logger import Logger
    from utils.visualizer import Visualizer
    print("✅ TurtleTrader modules succesvol geïmporteerd")
except ImportError as e:
    display_error(e)
    raise ImportError("Kon TurtleTrader modules niet importeren. Controleer het project pad.")

print("Setup voltooid. Klaar om te starten met validatie.")

Project pad: C:\Users\basti\PycharmProjects\TurtleTrader
✅ TurtleTrader modules succesvol geïmporteerd
Setup voltooid. Klaar om te starten met validatie.


# SECTIE 2: Configuratie Laden
# ===========================
# Laad de TurtleTrader configuratie uit het JSON-bestand

In [2]:


def load_config(config_path):
    """
    Laad configuratie uit JSON bestand met foutafhandeling

    Parameters:
    -----------
    config_path : str
        Pad naar het configuratiebestand

    Returns:
    --------
    dict
        Geladen configuratie als dictionary
    """
    try:
        with open(config_path, 'r') as f:
            config = json.load(f)
        return config
    except FileNotFoundError:
        raise FileNotFoundError(f"Configuratiebestand niet gevonden op {config_path}. Controleer het pad.")
    except json.JSONDecodeError:
        raise ValueError(f"Ongeldige JSON in configuratiebestand {config_path}. Controleer het bestandsformaat.")

try:
    # Configuratie laden
    config_path = os.path.join(project_path, 'config', 'turtle_settings.json')
    print(f"Configuratie zoeken op: {config_path}")

    # Controleer of het bestand bestaat
    if not os.path.exists(config_path):
        print(f"⚠️ Configuratiebestand niet gevonden op de verwachte locatie.")
        print(f"Laten we de mapstructuur controleren:")

        # Mapstructuur weergeven voor debugging
        project_contents = os.listdir(project_path)
        print(f"Inhoud van {project_path}:")
        for item in sorted(project_contents):
            if os.path.isdir(os.path.join(project_path, item)):
                print(f"  📁 {item}/")
            else:
                print(f"  📄 {item}")

    # Configuratie laden
    config = load_config(config_path)

    # Configuratie weergeven
    print("\nTurtleTrader Configuratie:")
    print(f"Broker: {config['mt5']['server']}")
    print(f"Login: {config['mt5']['login']}")
    print(f"MT5 pad: {config['mt5']['mt5_pathway']}")
    print(f"Symbolen: {config['mt5']['symbols']}")
    print(f"Timeframe: {config['mt5']['timeframe']}")
    print(f"Risico per trade: {config['mt5']['risk_per_trade'] * 100}%")

    if 'symbol_mapping' in config['mt5']:
        print(f"Symbool mapping: {config['mt5']['symbol_mapping']}")

    # Voor makkelijke referentie in het notebook
    mt5_config = config['mt5']

    # Definieer timeframe mapping voor later gebruik
    timeframe_dict = {
        "M1": mt5.TIMEFRAME_M1,
        "M5": mt5.TIMEFRAME_M5,
        "M15": mt5.TIMEFRAME_M15,
        "M30": mt5.TIMEFRAME_M30,
        "H1": mt5.TIMEFRAME_H1,
        "H4": mt5.TIMEFRAME_H4,
        "D1": mt5.TIMEFRAME_D1,
        "W1": mt5.TIMEFRAME_W1,
        "MN1": mt5.TIMEFRAME_MN1
    }

    # Gebruik de timeframe uit de configuratie
    if 'timeframe' in mt5_config and mt5_config['timeframe'] in timeframe_dict:
        mt5_timeframe = timeframe_dict[mt5_config['timeframe']]
        print(f"Gebruik timeframe: {mt5_config['timeframe']} (MT5 constant: {mt5_timeframe})")
    else:
        mt5_timeframe = mt5.TIMEFRAME_D1
        print(f"Gebruik standaard timeframe: D1 (MT5 constant: {mt5_timeframe})")

    print("\nConfiguratie succesvol geladen.")
except Exception as e:
    display_error(e)
    raise Exception("Kon configuratie niet laden. Zie details hierboven.")
# Beschikbare symbolen ophalen en printen
try:
    if not mt5.initialize():
        raise RuntimeError("MT5 initialisatie mislukt.")

    print("\nBeschikbare symbolen bij broker:")
    symbols = mt5.symbols_get()
    for symbol in symbols:
        print(symbol.name)

    mt5.shutdown()
except Exception as e:
    print(f"Fout bij ophalen van symbolen: {e}")


Configuratie zoeken op: C:\Users\basti\PycharmProjects\TurtleTrader\config\turtle_settings.json

TurtleTrader Configuratie:
Broker: FTMO-Demo2
Login: 1520533067
MT5 pad: C:\Program Files\FTMO MetaTrader 5\terminal64.exe
Symbolen: ['XAUUSD', 'US30']
Timeframe: H4
Risico per trade: 1.5%
Symbool mapping: {'US30': 'US30.cash'}
Gebruik timeframe: H4 (MT5 constant: 16388)

Configuratie succesvol geladen.

Beschikbare symbolen bij broker:
EURUSD
GBPUSD
USDCHF
USDJPY
USDCAD
AUDUSD
AUDNZD
AUDCAD
AUDCHF
AUDJPY
CHFJPY
EURGBP
EURAUD
EURCHF
EURJPY
EURNZD
EURCAD
GBPCHF
GBPJPY
CADCHF
CADJPY
GBPAUD
GBPCAD
GBPNZD
NZDCAD
NZDCHF
NZDJPY
NZDUSD
GER40.cash
UK100.cash
US100.cash
US30.cash
US500.cash
AUS200.cash
EU50.cash
FRA40.cash
HK50.cash
JP225.cash
SPN35.cash
US2000.cash
N25.cash
UKOIL.cash
USOIL.cash
XAUUSD
XAUAUD
XAUEUR
XAGAUD
XAGEUR
XAGUSD
XPDUSD
XPTUSD
ADAUSD
BTCUSD
DASHUSD
DOGEUSD
DOTUSD
ETHUSD
LTCUSD
NEOUSD
XMRUSD
XRPUSD
EURCZK
EURHUF
EURNOK
EURPLN
USDCZK
USDHKD
USDHUF
USDILS
USDMXN
USDNOK
USDPLN
U

# SECTIE 3: MT5 Verbinding Testen
# ==============================
# Test de verbinding met MetaTrader 5 en verkrijg marktgegevens

In [3]:


def test_mt5_connection(config):
    """
    Test de verbinding met MetaTrader 5 en geef informatie weer over het account en de beschikbare symbolen

    Parameters:
    -----------
    config : dict
        MT5 configuratie-instellingen

    Returns:
    --------
    MT5Connector of None
        Een geïnitialiseerde connector bij succes, None bij mislukking
    """
    print("\n" + "="*50)
    print("MT5 VERBINDINGSTEST")
    print("="*50 + "\n")

    try:
        # Initialiseer connector
        connector = MT5Connector(config)
        start_time = datetime.now()
        connected = connector.connect()
        connection_time = (datetime.now() - start_time).total_seconds()

        if connected:
            print(f"✅ MT5 verbinding succesvol! (Verbindingstijd: {connection_time:.2f} seconden)")

            # Account informatie
            account_info = connector.get_account_info()
            print(f"\nAccount Informatie:")
            print(f"  Server: {mt5.account_info().server}")
            print(f"  Balance: ${account_info.get('balance', 'Onbekend'):,.2f}")
            print(f"  Equity: ${account_info.get('equity', 'Onbekend'):,.2f}")
            print(f"  Margin: ${account_info.get('margin', 'Onbekend'):,.2f}")
            print(f"  Free Margin: ${account_info.get('free_margin', 'Onbekend'):,.2f}")
            print(f"  Profit: ${account_info.get('profit', 'Onbekend'):,.2f}")

            # Test symbolen
            print("\nSymbool Status:")
            symbol_table = []

            for symbol in config['symbols']:
                try:
                    # Pas symbool mapping toe indien nodig
                    mapped_symbol = symbol
                    if 'symbol_mapping' in config and symbol in config['symbol_mapping']:
                        mapped_symbol = config['symbol_mapping'][symbol]

                    # Haal symbool info op
                    symbol_info = mt5.symbol_info(mapped_symbol)
                    if symbol_info is not None:
                        print(f"  ✅ {symbol} (als {mapped_symbol}): Beschikbaar")
                        tick = connector.get_symbol_tick(symbol)
                        if tick is not None:
                            spread = (tick.ask - tick.bid) / tick.bid * 100 if tick.bid > 0 else 0
                            print(f"     Bid: {tick.bid:.5f}, Ask: {tick.ask:.5f}, Spread: {spread:.4f}%")

                            # Verzamel symbool informatie voor tabel
                            symbol_table.append({
                                'Symbool': symbol,
                                'Mapping': mapped_symbol if mapped_symbol != symbol else '-',
                                'Bid': tick.bid,
                                'Ask': tick.ask,
                                'Spread (%)': spread,
                                'Volume': tick.volume,
                                'Tijd': datetime.fromtimestamp(tick.time)
                            })
                    else:
                        print(f"  ❌ {symbol} (als {mapped_symbol}): Niet beschikbaar")
                except Exception as e:
                    print(f"  ❌ {symbol}: Fout bij ophalen info: {e}")

            # Toon samenvattingstabel indien er symbolen zijn
            if symbol_table:
                print("\nSymbool Overzicht:")
                symbol_df = pd.DataFrame(symbol_table)
                display(symbol_df)

            return connector
        else:
            print(f"❌ Kon geen verbinding maken met MT5.")
            print(f"Controleer of MT5 is geïnstalleerd op het opgegeven pad en dat de inloggegevens correct zijn.")
            return None
    except Exception as e:
        display_error(e)
        return None

# Voer verbindingstest uit
print("Start MT5 verbindingstest...")
connector = test_mt5_connection(mt5_config)

if connector is not None:
    print("\nVerbinding met MT5 succesvol tot stand gebracht. Klaar om verder te gaan met data-analyse.")
else:
    print("\n⚠️ MT5 verbinding niet tot stand gebracht. Controleer bovenstaande foutmeldingen.")
    print("Opmerking: Sommige validaties kunnen nog steeds worden uitgevoerd zonder live verbinding.")

Start MT5 verbindingstest...

MT5 VERBINDINGSTEST

Symbool mapping toegepast: US30 -> US30.cash
Verbonden met MT5: FTMO-Demo2
✅ MT5 verbinding succesvol! (Verbindingstijd: 0.00 seconden)

Account Informatie:
  Server: FTMO-Demo2
  Balance: $79,997.56
  Equity: $79,997.56
  Margin: $0.00
  Free Margin: $79,997.56
  Profit: $0.00

Symbool Status:
  ✅ XAUUSD (als XAUUSD): Beschikbaar
     Bid: 2858.42000, Ask: 2859.10000, Spread: 0.0238%
  ✅ US30 (als US30.cash): Beschikbaar
     Bid: 43772.83000, Ask: 43775.16000, Spread: 0.0053%

Symbool Overzicht:


,Symbool,Mapping,Bid,Ask,Spread (%),Volume,Tijd
0,XAUUSD,-,2858.42,2859.10,0.023789,0,2025-03-01 00:49:54
1,US30,US30.cash,43772.83,43775.16,0.005323,0,2025-03-01 00:49:48



Verbinding met MT5 succesvol tot stand gebracht. Klaar om verder te gaan met data-analyse.


# SECTIE 4: Historische Data Ophalen en Valideren
# ==============================================
# Haal historische data op voor elk symbool en voer uitgebreide validatie uit

In [4]:
# SECTIE 4: Intelligente Data Validatie voor Alle Symbolen
# ========================================================

def fetch_and_validate_data(connector, symbol, timeframe, bars_count, check_outliers=True):
    """
    Haal historische data op en voer uitgebreide validatie uit

    Parameters:
    -----------
    connector : MT5Connector
        Verbinding met MetaTrader 5
    symbol : str
        Het te analyseren symbool
    timeframe : int
        MT5 timeframe constante
    bars_count : int
        Aantal candles om op te halen
    check_outliers : bool, optional
        Of outliers gedetecteerd moeten worden

    Returns:
    --------
    pandas.DataFrame of None
        Gevalideerde historische data of None bij fouten
    """
    print("\n" + "="*50)
    print(f"DATA VALIDATIE: {symbol}")
    print("="*50 + "\n")

    if not connector or not connector.connected:
        print("❌ Geen verbinding met MT5. Kan geen data ophalen.")
        return None

    try:
        # Data ophalen met timing
        start_time = datetime.now()
        df = connector.get_historical_data(symbol, timeframe, bars_count)
        fetch_time = (datetime.now() - start_time).total_seconds()

        if df is None or df.empty:
            print(f"❌ Geen data opgehaald voor {symbol}")
            return None

        print(f"✅ Data succesvol opgehaald! ({len(df)} candles, tijd: {fetch_time:.2f} seconden)")
        print(f"Periode: {df['time'].min()} tot {df['time'].max()}")

        # Basis data informatie
        print(f"\nDataframe info: {df.shape[0]} rijen × {df.shape[1]} kolommen")
        print(f"Kolommen: {', '.join(df.columns.tolist())}")
        print("\nDataframe head (eerste 5 rijen):")
        display(df.head())

        # ===== DATA INTEGRITEITSCONTROLES =====
        print("\n" + "-"*30)
        print("DATA INTEGRITEITSCONTROLE")
        print("-"*30)

        validation_issues = []

        # 1. Controleer op NaN-waarden
        nan_counts = df.isna().sum()
        if nan_counts.sum() > 0:
            print("⚠️ NaN-waarden gevonden:")
            nan_cols = nan_counts[nan_counts > 0]
            for col, count in nan_cols.items():
                pct = count / len(df) * 100
                print(f"  - {col}: {count} waarden ({pct:.2f}%)")
                validation_issues.append(f"NaN in {col}: {count} waarden")
        else:
            print("✅ Geen NaN-waarden gevonden")

        # 2. Controleer op duplicaten
        if df['time'].duplicated().any():
            duplicate_count = df['time'].duplicated().sum()
            dup_pct = duplicate_count / len(df) * 100
            print(f"⚠️ {duplicate_count} dubbele tijdstempels gevonden ({dup_pct:.2f}%)")
            validation_issues.append(f"Duplicaten: {duplicate_count} tijdstempels")

            # Toon enkele voorbeelden van duplicaten
            dup_times = df[df['time'].duplicated(keep=False)]['time'].sort_values().unique()
            if len(dup_times) > 0:
                print("  Voorbeelden van dubbele tijdstempels:")
                for i, time in enumerate(dup_times[:3]):
                    print(f"  - {time}")
                if len(dup_times) > 3:
                    print(f"    ... en {len(dup_times) - 3} meer")
        else:
            print("✅ Geen dubbele tijdstempels gevonden")

        # 3. Controleer op gaten in de tijdreeks
        df = df.sort_values('time').reset_index(drop=True)
        df['time_diff'] = df['time'].diff()

        if timeframe == mt5.TIMEFRAME_D1:
            # Voor dagelijkse data, controleer op ontbrekende handelsdagen (zonder weekenden)
            df['weekday'] = df['time'].dt.dayofweek
            gaps = df[(df['time_diff'] > timedelta(days=1)) & (df['weekday'] != 0)]

            if len(gaps) > 0:
                gap_pct = len(gaps) / len(df) * 100
                print(f"⚠️ {len(gaps)} gaten in de tijdreeks gevonden ({gap_pct:.2f}%, exclusief weekenden)")
                validation_issues.append(f"Tijdreeksgaten: {len(gaps)} gaten")

                print("  Voorbeelden van gaten (exclusief weekenden):")
                sample_gaps = gaps.head(min(3, len(gaps)))
                for _, row in sample_gaps.iterrows():
                    print(f"  - Gap van {row['time_diff'].days} dagen tussen {row['time'] - row['time_diff']} en {row['time']}")
            else:
                print("✅ Geen onverwachte gaten in de tijdreeks")
        else:
            # Voor andere timeframes een eenvoudigere controle
            expected_diff = {
                mt5.TIMEFRAME_M1: timedelta(minutes=1),
                mt5.TIMEFRAME_M5: timedelta(minutes=5),
                mt5.TIMEFRAME_M15: timedelta(minutes=15),
                mt5.TIMEFRAME_M30: timedelta(minutes=30),
                mt5.TIMEFRAME_H1: timedelta(hours=1),
                mt5.TIMEFRAME_H4: timedelta(hours=4),
                mt5.TIMEFRAME_W1: timedelta(days=7),
                mt5.TIMEFRAME_MN1: timedelta(days=28)  # Benadering
            }

            if timeframe in expected_diff:
                expected = expected_diff[timeframe]
                # We slaan weekenden etc. over, dus check alleen items die >2x de expected diff zijn
                significant_gaps = df[df['time_diff'] > expected * 2].copy()

                if len(significant_gaps) > 0:
                    gap_pct = len(significant_gaps) / len(df) * 100
                    print(f"⚠️ {len(significant_gaps)} significante gaten in de tijdreeks gevonden ({gap_pct:.2f}%)")
                    validation_issues.append(f"Tijdreeksgaten: {len(significant_gaps)} gaten")

                    print("  Voorbeelden van significante gaten:")
                    sample_gaps = significant_gaps.head(min(3, len(significant_gaps)))
                    for _, row in sample_gaps.iterrows():
                        print(f"  - Gap van {row['time_diff']} tussen {row['time'] - row['time_diff']} en {row['time']}")
                else:
                    print("✅ Geen significante gaten in de tijdreeks")
            else:
                print("ℹ️ Gatencontrole niet beschikbaar voor deze timeframe")

        # 4. Controleer op outliers indien gevraagd
        if check_outliers:
            print("\n" + "-"*30)
            print("OUTLIER DETECTIE")
            print("-"*30)

            # Z-score methode
            z_threshold = 3.0
            numeric_cols = ['open', 'high', 'low', 'close', 'tick_volume']
            outlier_counts = {}

            for col in numeric_cols:
                if col in df.columns:
                    z_scores = np.abs(stats.zscore(df[col].dropna()))
                    outliers = np.where(z_scores > z_threshold)[0]

                    if len(outliers) > 0:
                        outlier_pct = len(outliers) / len(df) * 100
                        print(f"⚠️ {col}: {len(outliers)} outliers gevonden ({outlier_pct:.2f}%)")
                        outlier_counts[col] = len(outliers)
                        validation_issues.append(f"Outliers in {col}: {len(outliers)} waarden")

                        # Toon enkele voorbeelden
                        if len(outliers) > 0:
                            outlier_indices = outliers[:min(3, len(outliers))]
                            print("  Voorbeelden van outliers:")
                            for idx in outlier_indices:
                                if idx < len(df):
                                    z_val = z_scores[idx]
                                    print(f"  - {df.iloc[idx]['time']}: {col}={df.iloc[idx][col]:.5f} (z-score: {z_val:.2f})")
                    else:
                        print(f"✅ {col}: Geen outliers gevonden")

            # Plot outliers als er zijn gevonden (optioneel)
            if any(outlier_counts.values()) and False:  # Set to True to enable plots
                plt.figure(figsize=(14, 8))
                plt.subplot(2, 1, 1)
                plt.plot(df['time'], df['close'], '-b', alpha=0.7, label='Sluitingskoers')

                # Highlight potential outliers in price
                if 'close' in outlier_counts and outlier_counts['close'] > 0:
                    z_scores = np.abs(stats.zscore(df['close'].dropna()))
                    outlier_mask = z_scores > z_threshold
                    plt.scatter(df.loc[outlier_mask, 'time'], df.loc[outlier_mask, 'close'],
                                color='red', s=50, label='Outliers')

                plt.title(f'Prijsdata met Outliers - {symbol}')
                plt.ylabel('Prijs')
                plt.legend()
                plt.grid(True, alpha=0.3)

                # Plot volume
                plt.subplot(2, 1, 2)
                plt.bar(df['time'], df['tick_volume'], color='blue', alpha=0.5, label='Volume')

                # Highlight potential outliers in volume
                if 'tick_volume' in outlier_counts and outlier_counts['tick_volume'] > 0:
                    z_scores = np.abs(stats.zscore(df['tick_volume'].dropna()))
                    outlier_mask = z_scores > z_threshold
                    plt.scatter(df.loc[outlier_mask, 'time'], df.loc[outlier_mask, 'tick_volume'],
                                color='red', s=50, label='Volume Outliers')

                plt.title('Volume met Outliers')
                plt.ylabel('Volume')
                plt.legend()
                plt.tight_layout()
                plt.show()

        # 5. Prijssprong detectie (extreme bewegingen)
        print("\n" + "-"*30)
        print("PRIJSSPRONG DETECTIE")
        print("-"*30)

        df['pct_change'] = df['close'].pct_change() * 100
        extreme_moves = df[abs(df['pct_change']) > 2.0]  # Bewegingen >2% als voorbeeld

        if len(extreme_moves) > 0:
            move_pct = len(extreme_moves) / len(df) * 100
            print(f"ℹ️ {len(extreme_moves)} significante prijsbewegingen (>2%) gevonden ({move_pct:.2f}%)")

            print("  Top 3 grootste bewegingen:")
            top_moves = extreme_moves.sort_values('pct_change', key=abs, ascending=False).head(3)
            for _, row in top_moves.iterrows():
                print(f"  - {row['time']}: {row['pct_change']:.2f}% ({row['close']:.5f})")
        else:
            print("ℹ️ Geen extreme prijsbewegingen (>2%) gevonden")

        # 6. Beschrijvende statistieken
        print("\n" + "-"*30)
        print("BESCHRIJVENDE STATISTIEKEN")
        print("-"*30)

        stats_df = df[['open', 'high', 'low', 'close', 'tick_volume']].describe()
        display(stats_df)

        # 7. Validatie samenvatting
        print("\n" + "-"*30)
        print("VALIDATIE SAMENVATTING")
        print("-"*30)

        if validation_issues:
            print(f"⚠️ {len(validation_issues)} potentiële validatie-problemen gevonden:")
            for issue in validation_issues:
                print(f"  - {issue}")
            print("\nOverweeg of deze problemen de strategie-evaluatie kunnen beïnvloeden.")
        else:
            print("✅ Geen validatie-problemen gevonden. Data lijkt van hoge kwaliteit.")

        return df
    except Exception as e:
        display_error(e)
        return None

# Historische gegevens ophalen en valideren voor alle symbolen
if connector:
    # Aantal candles om op te halen (2 jaar dagelijkse data voor backtest)
    days_back = 365 * 2
    bars_count = days_back
    if mt5_config['timeframe'] == 'H1':
        bars_count = days_back * 24
    elif mt5_config['timeframe'] == 'H4':
        bars_count = days_back * 6

    print("\n" + "="*50)
    print("INTELLIGENTE DATA VALIDATIE VOOR ALLE SYMBOLEN")
    print("="*50 + "\n")

    # Gegevens voor alle symbolen ophalen en valideren
    symbol_data = {}
    validation_summaries = []

    # Loop door alle symbolen
    for symbol in mt5_config['symbols']:
        print(f"\n\n{'#'*70}")
        print(f"### VALIDATIE VOOR {symbol} ###")
        print(f"{'#'*70}\n")

        df = fetch_and_validate_data(connector, symbol, mt5_timeframe, bars_count)

        if df is not None:
            symbol_data[symbol] = df

            # Verzamel validatie-metrics voor vergelijking
            metrics = {
                'symbol': symbol,
                'candles': len(df),
                'period_start': df['time'].min(),
                'period_end': df['time'].max(),
                'avg_price': df['close'].mean(),
                'volatility': df['close'].pct_change().std() * 100,  # Dagelijkse volatiliteit in %
                'gaps': len(df[(df['time_diff'] > timedelta(days=1)) & (df['weekday'] != 0)]) if 'weekday' in df.columns else 'N/A',
                'outliers_close': len(df[abs(stats.zscore(df['close'])) > 3]) if len(df) > 30 else 'N/A',
                'big_moves': len(df[abs(df['pct_change']) > 2.0]) if 'pct_change' in df.columns else 'N/A',
                'avg_volume': df['tick_volume'].mean()
            }
            validation_summaries.append(metrics)

            print(f"\n✅ {symbol} data succesvol opgehaald en gevalideerd.")
        else:
            print(f"\n❌ Kon geen data valideren voor {symbol}.")

    # Als we minstens één symbool hebben verwerkt, toon dan een vergelijkend overzicht
    if validation_summaries:
        print("\n" + "="*80)
        print("VERGELIJKEND OVERZICHT VAN ALLE SYMBOLEN")
        print("="*80 + "\n")

        # Converteer naar DataFrame voor mooie weergave
        summary_df = pd.DataFrame(validation_summaries).set_index('symbol')

        # Bereken extra vergelijkingsmetrieken
        if len(summary_df) > 1:
            # Normaliseer volatiliteit (hoger = meer volatiel)
            max_vol = summary_df['volatility'].max()
            if max_vol > 0:
                summary_df['rel_volatility'] = summary_df['volatility'] / max_vol

            # Bereken een 'trading potential score' (hoger = beter)
            summary_df['trading_score'] = 0.0
            for idx, row in summary_df.iterrows():
                # Base score op volatiliteit en big moves, indien beschikbaar
                vol_score = min(row['volatility'] * 10, 100) if isinstance(row['volatility'], (int, float)) else 50
                big_moves_score = min(row['big_moves'] * 20, 100) if isinstance(row['big_moves'], (int, float)) and row['big_moves'] != 'N/A' else 50
                summary_df.at[idx, 'trading_score'] = (vol_score + big_moves_score) / 2

        # Print het vergelijkend overzicht
        display(summary_df)

        # Visualiseer prijsbewegingen van alle symbolen voor vergelijking
        if len(symbol_data) > 1:
            plt.figure(figsize=(14, 7))

            # Normaliseer prijzen voor betere vergelijking
            for symbol, data in symbol_data.items():
                # Bereken genormaliseerde prijs (eerste waarde = 100)
                normalized = data['close'] / data['close'].iloc[0] * 100
                plt.plot(data['time'], normalized, label=symbol)

            plt.title('Vergelijking van Genormaliseerde Prijzen (Startwaarde = 100)')
            plt.xlabel('Datum')
            plt.ylabel('Genormaliseerde Prijs')
            plt.legend()
            plt.grid(True, alpha=0.3)
            plt.show()

            # Volatiliteit vergelijking
            plt.figure(figsize=(10, 6))
            volatilities = [df['close'].pct_change().std() * 100 for df in symbol_data.values()]
            plt.bar(symbol_data.keys(), volatilities)
            plt.title('Volatiliteit Vergelijking (Dagelijkse standaarddeviatie in %)')
            plt.ylabel('Volatiliteit (%)')
            plt.grid(True, alpha=0.3, axis='y')
            plt.show()

        # Toon aanbeveling op basis van analyse
        if len(summary_df) > 1 and 'trading_score' in summary_df.columns:
            best_symbol = summary_df['trading_score'].idxmax()
            print(f"\n🔍 Analyse: Op basis van de historische data lijkt {best_symbol} het meest geschikt voor de TurtleTrader strategie")
            print(f"   vanwege de combinatie van volatiliteit en significante prijsbewegingen.")

    # Aantal symbolen waarvoor we succesvolle gegevens hebben verzameld
    success_count = len(symbol_data)
    total_count = len(mt5_config['symbols'])

    print(f"\nSuccesvol gegevens verzameld voor {success_count}/{total_count} symbolen.")
else:
    print("Kon geen verbinding maken met MT5. Kan geen historische data ophalen.")


INTELLIGENTE DATA VALIDATIE VOOR ALLE SYMBOLEN



######################################################################
### VALIDATIE VOOR XAUUSD ###
######################################################################


DATA VALIDATIE: XAUUSD

✅ Data succesvol opgehaald! (4380 candles, tijd: 0.41 seconden)
Periode: 2022-05-03 20:00:00 tot 2025-02-28 16:00:00

Dataframe info: 4380 rijen × 8 kolommen
Kolommen: time, open, high, low, close, tick_volume, spread, real_volume

Dataframe head (eerste 5 rijen):


,time,open,high,low,close,tick_volume,spread,real_volume
0,2022-05-03 20:00:00,1869.90,1871.84,1865.28,1867.89,7238,7,0
1,2022-05-04 00:00:00,1867.98,1869.43,1864.94,1865.92,2778,7,0
2,2022-05-04 04:00:00,1865.88,1866.22,1861.61,1865.66,5234,7,0
3,2022-05-04 08:00:00,1865.66,1872.04,1861.38,1868.47,7965,7,0
4,2022-05-04 12:00:00,1868.50,1871.62,1863.69,1867.50,9801,7,0



------------------------------
DATA INTEGRITEITSCONTROLE
------------------------------
✅ Geen NaN-waarden gevonden
✅ Geen dubbele tijdstempels gevonden
⚠️ 149 significante gaten in de tijdreeks gevonden (3.40%)
  Voorbeelden van significante gaten:
  - Gap van 2 days 04:00:00 tussen 2022-05-06 20:00:00 en 2022-05-09 00:00:00
  - Gap van 2 days 04:00:00 tussen 2022-05-13 20:00:00 en 2022-05-16 00:00:00
  - Gap van 2 days 04:00:00 tussen 2022-05-20 20:00:00 en 2022-05-23 00:00:00

------------------------------
OUTLIER DETECTIE
------------------------------
✅ open: Geen outliers gevonden
✅ high: Geen outliers gevonden
✅ low: Geen outliers gevonden
✅ close: Geen outliers gevonden
⚠️ tick_volume: 56 outliers gevonden (1.28%)
  Voorbeelden van outliers:
  - 2023-03-13 16:00:00: tick_volume=59860.00000 (z-score: 3.15)
  - 2023-03-16 16:00:00: tick_volume=59810.00000 (z-score: 3.14)
  - 2023-10-12 16:00:00: tick_volume=59585.00000 (z-score: 3.13)

------------------------------
PRIJSSPRONG

,open,high,low,close,tick_volume
count,4380.000000,4380.000000,4380.000000,4380.000000,4380.000000
mean,2106.004623,2111.212146,2100.878788,2106.177438,19772.573744
std,338.261432,339.187279,337.250060,338.415403,12737.761889
min,1618.270000,1625.570000,1614.750000,1618.270000,72.000000
25%,1851.052500,1854.337500,1846.090000,1850.890000,10462.000000
50%,1983.410000,1987.175000,1978.785000,1983.485000,16847.500000
75%,2359.957500,2365.455000,2354.890000,2360.160000,26261.750000
max,2954.460000,2956.150000,2945.230000,2954.440000,88227.000000



------------------------------
VALIDATIE SAMENVATTING
------------------------------
⚠️ 2 potentiële validatie-problemen gevonden:
  - Tijdreeksgaten: 149 gaten
  - Outliers in tick_volume: 56 waarden

Overweeg of deze problemen de strategie-evaluatie kunnen beïnvloeden.

✅ XAUUSD data succesvol opgehaald en gevalideerd.


######################################################################
### VALIDATIE VOOR US30 ###
######################################################################


DATA VALIDATIE: US30

✅ Data succesvol opgehaald! (4380 candles, tijd: 0.40 seconden)
Periode: 2022-04-29 00:00:00 tot 2025-02-28 16:00:00

Dataframe info: 4380 rijen × 8 kolommen
Kolommen: time, open, high, low, close, tick_volume, spread, real_volume

Dataframe head (eerste 5 rijen):


...


------------------------------
DATA INTEGRITEITSCONTROLE
------------------------------
✅ Geen NaN-waarden gevonden
✅ Geen dubbele tijdstempels gevonden
⚠️ 150 significante gaten in de tijdreeks gevonden (3.42%)
  Voorbeelden van significante gaten:
  - Gap van 2 days 04:00:00 tussen 2022-04-29 20:00:00 en 2022-05-02 00:00:00
  - Gap van 2 days 04:00:00 tussen 2022-05-06 20:00:00 en 2022-05-09 00:00:00
  - Gap van 2 days 04:00:00 tussen 2022-05-13 20:00:00 en 2022-05-16 00:00:00

------------------------------
OUTLIER DETECTIE
------------------------------
✅ open: Geen outliers gevonden
✅ high: Geen outliers gevonden
✅ low: Geen outliers gevonden
✅ close: Geen outliers gevonden
⚠️ tick_volume: 43 outliers gevonden (0.98%)
  Voorbeelden van outliers:
  - 2022-10-13 16:00:00: tick_volume=61246.00000 (z-score: 3.61)
  - 2022-10-14 16:00:00: tick_volume=58004.00000 (z-score: 3.36)
  - 2022-10-21 16:00:00: tick_volume=61001.00000 (z-score: 3.59)

------------------------------
PRIJSSPRONG

,open,high,low,close,tick_volume
count,4380.000000,4380.000000,4380.000000,4380.000000,4380.000000
mean,36363.792502,36443.994215,36281.696977,36366.120817,15015.956621
std,4152.674725,4143.816711,4161.179626,4153.196053,12805.248379
min,28621.700000,28834.200000,28586.200000,28620.200000,1.000000
25%,33200.525000,33286.350000,33115.000000,33198.750000,5205.000000
50%,34617.800000,34665.600000,34562.250000,34619.950000,9876.500000
75%,39316.825000,39393.250000,39263.350000,39317.375000,22251.500000
max,45016.860000,45079.100000,44994.060000,45016.860000,74960.000000



------------------------------
VALIDATIE SAMENVATTING
------------------------------
⚠️ 2 potentiële validatie-problemen gevonden:
  - Tijdreeksgaten: 150 gaten
  - Outliers in tick_volume: 43 waarden

Overweeg of deze problemen de strategie-evaluatie kunnen beïnvloeden.

✅ US30 data succesvol opgehaald en gevalideerd.

VERGELIJKEND OVERZICHT VAN ALLE SYMBOLEN



,candles,period_start,period_end,avg_price,volatility,gaps,outliers_close,big_moves,avg_volume,rel_volatility,trading_score
symbol,,,,,,,,,,,
XAUUSD,4380,2022-05-03 20:00:00,2025-02-28 16:00:00,2106.177438,0.353229,N/A,0,5,19772.573744,0.953099,51.766145
US30,4380,2022-04-29 00:00:00,2025-02-28 16:00:00,36366.120817,0.370611,N/A,0,15,15015.956621,1.000000,51.853055



🔍 Analyse: Op basis van de historische data lijkt US30 het meest geschikt voor de TurtleTrader strategie
   vanwege de combinatie van volatiliteit en significante prijsbewegingen.

Succesvol gegevens verzameld voor 2/2 symbolen.


# SECTIE 5: Strategie Signaalvalidatie
# ====================================
# Valideer de signaalberekening van de TurtleStrategy en visualiseer entry/exit punten

In [5]:
# SECTIE 5: Strategie Signaalvalidatie
# ====================================
# Valideer de signaalberekening van de TurtleStrategy en visualiseer entry/exit punten

def validate_strategy_signals(df, symbol, entry_period=20, exit_period=10, atr_period=20, atr_multiplier=2):
    """
    Valideer de signaalberekening van de TurtleStrategy en visualiseer resultaten

    Parameters:
    -----------
    df : pandas.DataFrame
        DataFrame met historische prijsgegevens
    symbol : str
        Handelssymbool
    entry_period : int
        Periode voor entry breakout (Donchian channel)
    exit_period : int
        Periode voor exit breakout (Donchian channel)
    atr_period : int
        Periode voor ATR berekening
    atr_multiplier : float
        Vermenigvuldiger voor ATR bij stop loss berekening

    Returns:
    --------
    pandas.DataFrame
        DataFrame met toegevoegde signalen en indicatoren
    """
    print("\n" + "="*50)
    print(f"VALIDATIE STRATEGIE SIGNALEN: {symbol}")
    print("="*50 + "\n")

    # Maak een kopie van het DataFrame om de originele data niet te wijzigen
    df_signals = df.copy()

    # Bereken ATR (Average True Range)
    high = df_signals['high']
    low = df_signals['low']
    close = df_signals['close'].shift(1)

    tr1 = high - low
    tr2 = abs(high - close)
    tr3 = abs(low - close)

    df_signals['tr'] = pd.DataFrame({'tr1': tr1, 'tr2': tr2, 'tr3': tr3}).max(axis=1)
    df_signals['atr'] = df_signals['tr'].rolling(window=atr_period).mean()

    # Bereken Donchian channels
    df_signals['high_entry'] = df_signals['high'].rolling(window=entry_period).max()
    df_signals['low_exit'] = df_signals['low'].rolling(window=exit_period).min()

    # Bereken EMA voor trend filter
    df_signals['ema_50'] = df_signals['close'].ewm(span=50, adjust=False).mean()
    df_signals['trend_bullish'] = df_signals['close'] > df_signals['ema_50']

    # Bereken entry en exit signalen
    # Entry: Prijs breekt boven hoogste high van entry_period (en trend bullish)
    df_signals['entry_signal'] = (df_signals['close'] > df_signals['high_entry'].shift(1)) & \
                                 (df_signals['atr'] > 0) & \
                                 (df_signals['trend_bullish'])

    # Exit: Prijs breekt onder laagste low van exit_period
    df_signals['exit_signal'] = (df_signals['close'] < df_signals['low_exit'].shift(1))

    # Simuleer positie management
    df_signals['position'] = 0
    df_signals['stop_loss'] = np.nan

    position = 0
    entry_price = 0
    stop_loss = 0

    # Loop door de data om posities te simuleren
    for i in range(1, len(df_signals)):
        # Vorige positie overnemen
        df_signals.at[df_signals.index[i], 'position'] = position

        # Entry signaal
        if position == 0 and df_signals['entry_signal'].iloc[i]:
            position = 1
            entry_price = df_signals['close'].iloc[i]
            stop_loss = entry_price - (atr_multiplier * df_signals['atr'].iloc[i])

            df_signals.at[df_signals.index[i], 'position'] = position
            df_signals.at[df_signals.index[i], 'stop_loss'] = stop_loss

        # Exit signaal of stop loss
        elif position == 1 and (df_signals['exit_signal'].iloc[i] or df_signals['low'].iloc[i] <= stop_loss):
            position = 0
            entry_price = 0
            stop_loss = 0

            df_signals.at[df_signals.index[i], 'position'] = position

    # Tel aantal signalen
    entry_count = df_signals['entry_signal'].sum()
    exit_count = df_signals['exit_signal'].sum()

    print(f"Aantal entry signalen: {entry_count}")
    print(f"Aantal exit signalen: {exit_count}")

    # Visualiseer signalen
    plt.figure(figsize=(15, 10))

    # Plot 1: Prijs en Donchian channels
    plt.subplot(2, 1, 1)
    plt.plot(df_signals['time'], df_signals['close'], label='Close', color='blue', alpha=0.5)
    plt.plot(df_signals['time'], df_signals['high_entry'], label='Entry Channel', color='green', linestyle='--')
    plt.plot(df_signals['time'], df_signals['low_exit'], label='Exit Channel', color='red', linestyle='--')
    plt.plot(df_signals['time'], df_signals['ema_50'], label='EMA-50 (Trendfilter)', color='purple', alpha=0.7)

    # Plot entry en exit signalen
    entry_points = df_signals[df_signals['entry_signal']]
    exit_points = df_signals[df_signals['exit_signal']]

    plt.scatter(entry_points['time'], entry_points['close'], marker='^', color='g', s=100, label='Entry Signalen')
    plt.scatter(exit_points['time'], exit_points['close'], marker='v', color='r', s=100, label='Exit Signalen')

    # Plot stop losses
    stops = df_signals[df_signals['stop_loss'].notna()]
    for idx, row in stops.iterrows():
        plt.plot([row['time'], row['time']], [row['close'], row['stop_loss']], 'r-', alpha=0.5)

    plt.title(f'Turtle Strategie Signalen voor {symbol}')
    plt.ylabel('Prijs')
    plt.legend()
    plt.grid(True, alpha=0.3)

    # Plot 2: Positie
    plt.subplot(2, 1, 2)
    plt.fill_between(df_signals['time'], df_signals['position'], color='lightgreen', step='post', alpha=0.5)
    plt.plot(df_signals['time'], df_signals['position'], drawstyle='steps-post', color='green', label='Positie (1=Long, 0=Geen)')

    plt.title('Positie Tijdlijn')
    plt.ylabel('Positie')
    plt.xlabel('Datum')
    plt.yticks([0, 1])
    plt.legend()
    plt.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    return df_signals

# Test de strategie op elk symbool
if 'symbol_data' in locals() and symbol_data:
    # Kies een symbool om gedetailleerd te analyseren (bij voorkeur XAUUSD op basis van eerdere analyse)
    focus_symbol = 'XAUUSD' if 'XAUUSD' in symbol_data else list(symbol_data.keys())[0]

    print(f"\nGedetailleerde signaalanalyse voor {focus_symbol}...")
    signals_df = validate_strategy_signals(symbol_data[focus_symbol], focus_symbol)

    # Korte samenvatting van signalen voor alle symbolen tonen
    print("\nSamenvatting signalen per symbool:")
    for symbol, df in symbol_data.items():
        if symbol != focus_symbol:  # We hebben focus_symbol al gedetailleerd geanalyseerd
            # Korte analyse zonder uitgebreide visualisatie
            temp_df = df.copy()
            temp_df['high_entry'] = temp_df['high'].rolling(window=20).max()
            temp_df['ema_50'] = temp_df['close'].ewm(span=50, adjust=False).mean()
            temp_df['trend_bullish'] = temp_df['close'] > temp_df['ema_50']
            temp_df['entry_signal'] = (temp_df['close'] > temp_df['high_entry'].shift(1)) & temp_df['trend_bullish']
            entry_count = temp_df['entry_signal'].sum()

            print(f"- {symbol}: {entry_count} potentiële entry signalen in de afgelopen 2 jaar")
else:
    print("Geen gevalideerde gegevens beschikbaar. Voer eerst de data validatie uit.")


Gedetailleerde signaalanalyse voor XAUUSD...

VALIDATIE STRATEGIE SIGNALEN: XAUUSD

Aantal entry signalen: 302
Aantal exit signalen: 324

Samenvatting signalen per symbool:
- US30: 304 potentiële entry signalen in de afgelopen 2 jaar


# SECTIE 6: Risicobeheer Validatie
# ================================
# Test de position sizing en risicobeheer componenten

In [6]:
# SECTIE 6: Risicobeheer Validatie
# ================================
# Test de position sizing en risicobeheer componenten

def validate_risk_management(symbol_data, symbol, account_balance=100000, risk_per_trade=0.01,
                            max_daily_drawdown=0.05, atr_multiplier=2):
    """
    Valideer de risicobeheer componenten voor de TurtleTrader strategie

    Parameters:
    -----------
    symbol_data : dict
        Dictionary met DataFrames voor elk symbool
    symbol : str
        Handelssymbool om te analyseren
    account_balance : float
        Initieel accountsaldo
    risk_per_trade : float
        Maximaal risico per trade als percentage van account
    max_daily_drawdown : float
        Maximale dagelijkse drawdown als percentage van account
    atr_multiplier : float
        Vermenigvuldiger voor ATR bij stop loss berekening
    """
    print("\n" + "="*50)
    print(f"RISICOBEHEER VALIDATIE: {symbol}")
    print("="*50 + "\n")

    if symbol not in symbol_data:
        print(f"Geen data beschikbaar voor {symbol}")
        return

    df = symbol_data[symbol].copy()

    # Bereken ATR voor stop loss bepaling
    high = df['high']
    low = df['low']
    close = df['close'].shift(1)

    tr1 = high - low
    tr2 = abs(high - close)
    tr3 = abs(low - close)

    df['tr'] = pd.DataFrame({'tr1': tr1, 'tr2': tr2, 'tr3': tr3}).max(axis=1)
    df['atr'] = df['tr'].rolling(window=20).mean()

    # Bereken signalen zoals in de vorige functie
    df['high_entry'] = df['high'].rolling(window=20).max()
    df['low_exit'] = df['low'].rolling(window=10).min()
    df['ema_50'] = df['close'].ewm(span=50, adjust=False).mean()
    df['trend_bullish'] = df['close'] > df['ema_50']

    df['entry_signal'] = (df['close'] > df['high_entry'].shift(1)) & \
                         (df['atr'] > 0) & \
                         (df['trend_bullish'])
    df['exit_signal'] = (df['close'] < df['low_exit'].shift(1))

    # Simuleer trading met positie grootte en risicobeheer
    df['position'] = 0
    df['stop_loss'] = np.nan
    df['position_size'] = np.nan
    df['risk_amount'] = np.nan
    df['equity'] = account_balance
    df['daily_pnl'] = 0

    position = 0
    entry_price = 0
    stop_loss = 0
    position_size = 0
    equity = account_balance
    daily_losses = 0
    last_date = None

    # Functie voor pip waarde berekening (vereenvoudigd)
    def calculate_pip_value(symbol, volume):
        if symbol == "EURUSD":
            return volume * 10  # Voor forex pairs met USD
        elif symbol == "XAUUSD":
            return volume * 10  # Voor goud (vereenvoudigd)
        elif symbol == "US30":
            return volume * 10  # Voor US30 index (vereenvoudigd)
        else:
            return volume * 10  # Algemene schatting

    # Functie voor pips at risk berekening (vereenvoudigd)
    def calculate_pips_at_risk(symbol, entry_price, stop_loss):
        difference = abs(entry_price - stop_loss)

        if symbol == "EURUSD":
            return difference / 0.0001  # 4 decimalen voor forex
        elif symbol == "XAUUSD":
            return difference / 0.1  # 1 decimaal voor goud
        elif symbol == "US30":
            return difference / 1.0  # 0 decimalen voor indices
        else:
            return difference / 0.0001  # Standaard

    # Loop door de data om trades te simuleren
    for i in range(1, len(df)):
        current_date = df['time'].iloc[i].date()

        # Reset dagelijkse verliezen bij nieuwe dag
        if last_date is not None and current_date != last_date:
            daily_losses = 0

        last_date = current_date

        # Update positie en equity van vorige tijdstap
        df.at[df.index[i], 'position'] = position
        df.at[df.index[i], 'equity'] = equity

        # Berekenen P&L als we een positie hebben
        if position > 0:
            current_pnl = (df['close'].iloc[i] - entry_price) * position_size * calculate_pip_value(symbol, 1)
            df.at[df.index[i], 'daily_pnl'] = current_pnl

            # Update equity
            equity = account_balance + current_pnl
            df.at[df.index[i], 'equity'] = equity

        # Entry signaal
        if position == 0 and df['entry_signal'].iloc[i]:
            entry_price = df['close'].iloc[i]
            stop_loss = entry_price - (atr_multiplier * df['atr'].iloc[i])

            # Bereken positiegrootte op basis van risico
            risk_amount = account_balance * risk_per_trade
            pips_at_risk = calculate_pips_at_risk(symbol, entry_price, stop_loss)

            # Bereken volume op basis van risico per pip
            pip_value = risk_amount / pips_at_risk if pips_at_risk > 0 else 0
            position_size = round(pip_value / calculate_pip_value(symbol, 1), 2) if pip_value > 0 else 0

            # Begrens positiegrootte (vereenvoudigd)
            position_size = max(0.01, min(position_size, 10.0))

            # Controleer dagelijks risico (FTMO regel)
            potential_loss = risk_amount
            if daily_losses + potential_loss > account_balance * max_daily_drawdown:
                # Skip trade als het dagelijks risico te hoog is
                print(f"⚠️ Trade op {df['time'].iloc[i]} overgeslagen vanwege dagelijks risico limiet")
                continue

            # Entry uitvoeren
            position = 1
            df.at[df.index[i], 'position'] = position
            df.at[df.index[i], 'stop_loss'] = stop_loss
            df.at[df.index[i], 'position_size'] = position_size
            df.at[df.index[i], 'risk_amount'] = risk_amount

        # Exit signaal of stop loss
        elif position == 1 and (df['exit_signal'].iloc[i] or df['low'].iloc[i] <= stop_loss):
            # Bereken P&L
            exit_price = df['close'].iloc[i] if df['exit_signal'].iloc[i] else stop_loss
            pnl = (exit_price - entry_price) * position_size * calculate_pip_value(symbol, 1)

            # Update account balance
            account_balance += pnl
            equity = account_balance

            # Update dagelijkse verliezen bij verlies
            if pnl < 0:
                daily_losses += abs(pnl)

            # Reset positie
            position = 0
            entry_price = 0
            stop_loss = 0
            position_size = 0

            df.at[df.index[i], 'position'] = position
            df.at[df.index[i], 'equity'] = equity

    # Bereken trading statistieken
    entry_indices = df[df['position'] > df['position'].shift(1)].index
    exit_indices = df[df['position'] < df['position'].shift(1)].index

    trades = []
    for i in range(min(len(entry_indices), len(exit_indices))):
        entry_idx = entry_indices[i]
        exit_idx = exit_indices[i] if i < len(exit_indices) else df.index[-1]

        if exit_idx <= entry_idx:
            continue

        entry_price = df.loc[entry_idx, 'close']
        exit_price = df.loc[exit_idx, 'close']
        position_size = df.loc[entry_idx, 'position_size']
        risk_amount = df.loc[entry_idx, 'risk_amount']

        pnl = (exit_price - entry_price) * position_size * calculate_pip_value(symbol, 1)
        risk_reward = pnl / risk_amount if risk_amount > 0 else 0

        trades.append({
            'entry_date': df.loc[entry_idx, 'time'],
            'exit_date': df.loc[exit_idx, 'time'],
            'duration': (df.loc[exit_idx, 'time'] - df.loc[entry_idx, 'time']).days,
            'entry_price': entry_price,
            'exit_price': exit_price,
            'position_size': position_size,
            'pnl': pnl,
            'risk_amount': risk_amount,
            'risk_reward': risk_reward
        })

    trades_df = pd.DataFrame(trades)

    # Toon risicobeheer statistieken
    print("\nRisicobeheer Statistieken:")
    print(f"Initieel Account Balance: ${account_balance:,.2f}")
    print(f"Eind Account Balance: ${equity:,.2f}")
    print(f"Winst/Verlies: ${equity - account_balance:,.2f} ({(equity/account_balance - 1) * 100:.2f}%)")

    if not trades_df.empty:
        print(f"\nAantal trades: {len(trades_df)}")
        print(f"Gemiddelde positiegrootte: {trades_df['position_size'].mean():.2f} lots")
        print(f"Gemiddeld risicobedrag per trade: ${trades_df['risk_amount'].mean():,.2f}")
        print(f"Gemiddelde trade duur: {trades_df['duration'].mean():.1f} dagen")

        # Winst/verlies statistieken
        winning_trades = trades_df[trades_df['pnl'] > 0]
        losing_trades = trades_df[trades_df['pnl'] <= 0]

        if len(winning_trades) > 0:
            win_rate = len(winning_trades) / len(trades_df)
            avg_win = winning_trades['pnl'].mean()
            print(f"\nWinrate: {win_rate:.2%}")
            print(f"Gemiddelde winnende trade: ${avg_win:,.2f}")

        if len(losing_trades) > 0:
            avg_loss = losing_trades['pnl'].mean()
            print(f"Gemiddelde verliezende trade: ${avg_loss:,.2f}")

        if len(winning_trades) > 0 and len(losing_trades) > 0:
            profit_factor = abs(winning_trades['pnl'].sum() / losing_trades['pnl'].sum()) if losing_trades['pnl'].sum() != 0 else float('inf')
            print(f"Profit Factor: {profit_factor:.2f}")

    # Visualiseer equity curve
    plt.figure(figsize=(15, 8))
    plt.plot(df['time'], df['equity'], label='Account Equity')

    # Markeer trades op de equity curve
    for i, trade in enumerate(trades):
        if trade['pnl'] > 0:
            color = 'green'
        else:
            color = 'red'

        plt.axvspan(trade['entry_date'], trade['exit_date'], alpha=0.2, color=color)

    plt.title(f'Equity Curve voor {symbol} met Risicobeheer')
    plt.xlabel('Datum')
    plt.ylabel('Equity ($)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

    return df, trades_df

# Test risicobeheer op het primaire symbool (XAUUSD)
if 'symbol_data' in locals() and symbol_data:
    # XAUUSD is hier onze focus vanwege de goede resultaten uit eerdere analyses
    focus_symbol = 'XAUUSD' if 'XAUUSD' in symbol_data else list(symbol_data.keys())[0]

    risk_df, trades_df = validate_risk_management(symbol_data, focus_symbol,
                                                account_balance=100000,
                                                risk_per_trade=0.01,  # 1% risico per trade
                                                max_daily_drawdown=0.05)  # 5% max dagelijkse drawdown (FTMO regel)

    if not trades_df.empty:
        # Toon top 5 meest winstgevende trades
        print("\nTop 5 Meest Winstgevende Trades:")
        display(trades_df.sort_values('pnl', ascending=False).head(5))

        # Toon top 5 meest verliesgevende trades
        print("\nTop 5 Meest Verliesgevende Trades:")
        display(trades_df.sort_values('pnl', ascending=True).head(5))
else:
    print("Geen gevalideerde gegevens beschikbaar. Voer eerst de data validatie uit.")


RISICOBEHEER VALIDATIE: XAUUSD


Risicobeheer Statistieken:
Initieel Account Balance: $101,587.38
Eind Account Balance: $101,587.38
Winst/Verlies: $0.00 (0.00%)

Aantal trades: 70
Gemiddelde positiegrootte: 0.53 lots
Gemiddeld risicobedrag per trade: $1,000.45
Gemiddelde trade duur: 5.0 dagen

Winrate: 35.71%
Gemiddelde winnende trade: $219.33
Gemiddelde verliezende trade: $-81.48
Profit Factor: 1.50

Top 5 Meest Winstgevende Trades:


,entry_date,exit_date,duration,entry_price,exit_price,position_size,pnl,risk_amount,risk_reward
45,2024-02-29 12:00:00,2024-03-18 04:00:00,17,2047.65,2147.35,0.81,807.570,994.901022,0.811709
35,2023-10-09 00:00:00,2023-10-31 20:00:00,22,1852.33,1983.69,0.61,801.296,990.057426,0.809343
47,2024-03-28 12:00:00,2024-04-22 08:00:00,24,2215.67,2359.53,0.47,676.142,1001.968962,0.674813
68,2025-01-30 08:00:00,2025-02-14 16:00:00,15,2774.27,2886.96,0.47,529.643,1011.604171,0.523567
53,2024-07-03 08:00:00,2024-07-18 20:00:00,15,2344.61,2444.68,0.51,510.357,1005.657847,0.507486



Top 5 Meest Verliesgevende Trades:


,entry_date,exit_date,duration,entry_price,exit_price,position_size,pnl,risk_amount,risk_reward
20,2023-02-01 20:00:00,2023-02-02 16:00:00,0,1950.39,1912.17,0.49,-187.278,996.277300,-0.187978
54,2024-07-30 20:00:00,2024-08-05 12:00:00,5,2410.58,2374.46,0.41,-148.092,1010.761417,-0.146515
29,2023-06-01 16:00:00,2023-06-02 16:00:00,1,1978.43,1952.26,0.56,-146.552,991.459566,-0.147814
8,2022-09-09 08:00:00,2022-09-13 12:00:00,4,1728.52,1703.85,0.56,-138.152,996.822126,-0.138592
7,2022-09-06 00:00:00,2022-09-06 12:00:00,0,1723.84,1707.71,0.70,-112.910,997.814866,-0.113157


# SECTIE 7: Verbeterde Backtesting
# ================================
# Implementeer en test strategie-verbeteringen op basis van eerdere validatie

In [7]:


def backtest_enhanced_strategy(df, symbol, account_balance=100000, risk_per_trade=0.01, atr_period=20,
                              entry_period=20, exit_period=10, atr_multiplier=2):
    """
    Vergelijk de originele Turtle Trading strategie met een verbeterde versie

    Parameters:
    -----------
    df : pandas.DataFrame
        DataFrame met historische prijsgegevens
    symbol : str
        Handelssymbool
    account_balance : float
        Initieel accountsaldo
    risk_per_trade : float
        Maximaal risico per trade als percentage van account
    atr_period : int
        Periode voor ATR berekening
    entry_period : int
        Periode voor entry breakout (Donchian channel)
    exit_period : int
        Periode voor exit breakout (Donchian channel)
    atr_multiplier : float
        Vermenigvuldiger voor ATR bij stop loss berekening

    Returns:
    --------
    tuple
        (original_results_df, enhanced_results_df, comparison_metrics)
    """
    print("\n" + "="*50)
    print("VERBETERDE BACKTESTING: ORIGINEEL VS. VERBETERD")
    print("="*50 + "\n")

    # Maak een kopie van het DataFrame
    df_original = df.copy()
    df_enhanced = df.copy()

    # --- STAP 1: BASISINDICATOREN BEREKENEN (beide strategieën delen deze) ---

    # Bereken ATR voor stop loss bepaling
    high = df['high']
    low = df['low']
    close = df['close'].shift(1)

    tr1 = high - low
    tr2 = abs(high - close)
    tr3 = abs(low - close)

    df_original['tr'] = pd.DataFrame({'tr1': tr1, 'tr2': tr2, 'tr3': tr3}).max(axis=1)
    df_original['atr'] = df_original['tr'].rolling(window=atr_period).mean()

    df_enhanced['tr'] = df_original['tr'].copy()
    df_enhanced['atr'] = df_original['atr'].copy()

    # Bereken Donchian channels
    df_original['high_entry'] = df_original['high'].rolling(window=entry_period).max()
    df_original['low_exit'] = df_original['low'].rolling(window=exit_period).min()
    df_original['ema_50'] = df_original['close'].ewm(span=50, adjust=False).mean()
    df_original['trend_bullish'] = df_original['close'] > df_original['ema_50']

    df_enhanced['high_entry'] = df_original['high_entry'].copy()
    df_enhanced['low_exit'] = df_original['low_exit'].copy()
    df_enhanced['ema_50'] = df_original['ema_50'].copy()
    df_enhanced['trend_bullish'] = df_original['trend_bullish'].copy()

    # --- STAP 2: ORIGINELE STRATEGIE ENTRY/EXIT SIGNALEN ---

    # Entry: Prijs breekt boven hoogste high van entry_period (en trend bullish)
    df_original['entry_signal'] = (df_original['close'] > df_original['high_entry'].shift(1)) & \
                                 (df_original['atr'] > 0) & \
                                 (df_original['trend_bullish'])

    # Exit: Prijs breekt onder laagste low van exit_period
    df_original['exit_signal'] = (df_original['close'] < df_original['low_exit'].shift(1))

    # --- STAP 3: VERBETERDE STRATEGIE ENTRY/EXIT SIGNALEN ---

    # VERBETERING 1: Betere entry filters
    # 1a. Vereist dat de slotkoers minstens 0.5% boven de breakout is
    breakout_threshold = 0.005  # 0.5%

    # 1b. Voeg volumebevestiging toe (volume > 50-daags gemiddelde)
    df_enhanced['vol_avg_50'] = df_enhanced['tick_volume'].rolling(window=50).mean()
    volume_filter = df_enhanced['tick_volume'] > df_enhanced['vol_avg_50']

    # 1c. Voeg trendsterkte filter toe (afstand tot EMA-50 als % van prijs)
    df_enhanced['trend_strength'] = (df_enhanced['close'] - df_enhanced['ema_50']) / df_enhanced['close']

    # Gecombineerd entry signaal met verbeterde filters
    df_enhanced['entry_signal'] = (df_enhanced['close'] > df_enhanced['high_entry'].shift(1) * (1 + breakout_threshold)) & \
                                 (df_enhanced['atr'] > 0) & \
                                 (df_enhanced['trend_bullish']) & \
                                 (volume_filter) & \
                                 (df_enhanced['trend_strength'] > 0)  # Positieve trend sterkte

    # VERBETERING 2: Verbeterde Exit Strategie
    # Standaard exit signaal (zoals origineel)
    df_enhanced['exit_signal'] = (df_enhanced['close'] < df_enhanced['low_exit'].shift(1))

    # Extra trailing stop bij sterke winsten (geïmplementeerd in backtesting loop)

    # --- STAP 4: BACKTEST UITVOEREN VOOR BEIDE STRATEGIEËN ---

    # Functie voor pip waarde berekening (vereenvoudigd)
    def calculate_pip_value(symbol, volume):
        if symbol == "EURUSD":
            return volume * 10
        elif symbol == "XAUUSD":
            return volume * 10
        elif symbol == "US30":
            return volume * 10
        else:
            return volume * 10

    # Functie voor pips at risk berekening
    def calculate_pips_at_risk(symbol, entry_price, stop_loss):
        difference = abs(entry_price - stop_loss)

        if symbol == "EURUSD":
            return difference / 0.0001
        elif symbol == "XAUUSD":
            return difference / 0.1
        elif symbol == "US30":
            return difference / 1.0
        else:
            return difference / 0.0001

    # Simulatie-resultaten voor beide strategieën
    results = {
        'original': {'equity': account_balance, 'trades': []},
        'enhanced': {'equity': account_balance, 'trades': []}
    }

    # --- LOOP VOOR ORIGINELE STRATEGIE ---
    df_original['position'] = 0
    df_original['stop_loss'] = np.nan
    df_original['position_size'] = np.nan
    df_original['equity'] = account_balance

    position = 0
    entry_price = 0
    stop_loss = 0
    position_size = 0
    equity = account_balance

    for i in range(1, len(df_original)):
        # Update positie en equity van vorige tijdstap
        df_original.at[df_original.index[i], 'position'] = position
        df_original.at[df_original.index[i], 'equity'] = equity

        # Berekenen P&L als we een positie hebben
        if position > 0:
            current_pnl = (df_original['close'].iloc[i] - entry_price) * position_size * calculate_pip_value(symbol, 1)

            # Update equity
            equity = account_balance + current_pnl
            df_original.at[df_original.index[i], 'equity'] = equity

        # Entry signaal
        if position == 0 and df_original['entry_signal'].iloc[i]:
            entry_price = df_original['close'].iloc[i]
            stop_loss = entry_price - (atr_multiplier * df_original['atr'].iloc[i])

            # Bereken positiegrootte op basis van risico
            risk_amount = account_balance * risk_per_trade
            pips_at_risk = calculate_pips_at_risk(symbol, entry_price, stop_loss)

            # Bereken volume op basis van risico per pip
            pip_value = risk_amount / pips_at_risk if pips_at_risk > 0 else 0
            position_size = round(pip_value / calculate_pip_value(symbol, 1), 2) if pip_value > 0 else 0

            # Begrens positiegrootte
            position_size = max(0.01, min(position_size, 10.0))

            # Entry uitvoeren
            position = 1
            df_original.at[df_original.index[i], 'position'] = position
            df_original.at[df_original.index[i], 'stop_loss'] = stop_loss
            df_original.at[df_original.index[i], 'position_size'] = position_size

            # Registreer trade
            results['original']['trades'].append({
                'entry_date': df_original['time'].iloc[i],
                'entry_price': entry_price,
                'position_size': position_size,
                'stop_loss': stop_loss,
                'risk_amount': risk_amount
            })

        # Exit signaal of stop loss
        elif position == 1 and (df_original['exit_signal'].iloc[i] or df_original['low'].iloc[i] <= stop_loss):
            # Bereken P&L
            exit_price = df_original['close'].iloc[i] if df_original['exit_signal'].iloc[i] else stop_loss
            pnl = (exit_price - entry_price) * position_size * calculate_pip_value(symbol, 1)

            # Update account balance
            account_balance += pnl
            equity = account_balance

            # Registreer exit in laatste trade
            if results['original']['trades']:
                results['original']['trades'][-1].update({
                    'exit_date': df_original['time'].iloc[i],
                    'exit_price': exit_price,
                    'pnl': pnl,
                    'duration': (df_original['time'].iloc[i] - results['original']['trades'][-1]['entry_date']).days
                })

            # Reset positie
            position = 0
            entry_price = 0
            stop_loss = 0
            position_size = 0

            df_original.at[df_original.index[i], 'position'] = position
            df_original.at[df_original.index[i], 'equity'] = equity

    # Sla eindequity op
    results['original']['equity'] = equity

    # --- LOOP VOOR VERBETERDE STRATEGIE ---
    account_balance = results['enhanced']['equity']  # Reset naar origineel startbedrag
    df_enhanced['position'] = 0
    df_enhanced['stop_loss'] = np.nan
    df_enhanced['position_size'] = np.nan
    df_enhanced['equity'] = account_balance
    df_enhanced['partial_exit'] = False  # Voor tracking van partiële exits

    position = 0
    entry_price = 0
    stop_loss = 0
    position_size = 0
    equity = account_balance
    partial_exit_done = False
    original_position_size = 0

    for i in range(1, len(df_enhanced)):
        # Update positie en equity van vorige tijdstap
        df_enhanced.at[df_enhanced.index[i], 'position'] = position
        df_enhanced.at[df_enhanced.index[i], 'equity'] = equity
        df_enhanced.at[df_enhanced.index[i], 'partial_exit'] = partial_exit_done

        # Berekenen P&L als we een positie hebben
        if position > 0:
            current_pnl = (df_enhanced['close'].iloc[i] - entry_price) * position_size * calculate_pip_value(symbol, 1)

            # Update equity
            equity = account_balance + current_pnl
            df_enhanced.at[df_enhanced.index[i], 'equity'] = equity

            # VERBETERING 3: Partiële Exit en Trailing Stop
            # Als prijs > entry_price + 1 ATR en geen partiële exit is gedaan
            atr_value = df_enhanced['atr'].iloc[i]

            if not partial_exit_done and position > 0 and df_enhanced['close'].iloc[i] > entry_price + atr_value:
                # Sluit 50% van de positie
                exit_size = position_size / 2
                position_size -= exit_size

                # Bereken winst op gesloten deel
                partial_exit_price = df_enhanced['close'].iloc[i]
                partial_pnl = (partial_exit_price - entry_price) * exit_size * calculate_pip_value(symbol, 1)

                # Update account balance
                account_balance += partial_pnl
                equity = account_balance + current_pnl/2  # Resterende positie

                # Verplaats stop loss naar break-even
                stop_loss = entry_price

                partial_exit_done = True
                df_enhanced.at[df_enhanced.index[i], 'partial_exit'] = True
                df_enhanced.at[df_enhanced.index[i], 'position_size'] = position_size
                df_enhanced.at[df_enhanced.index[i], 'stop_loss'] = stop_loss
                df_enhanced.at[df_enhanced.index[i], 'equity'] = equity

                # Noteer de partiële exit in het trade record
                if results['enhanced']['trades']:
                    results['enhanced']['trades'][-1].update({
                        'partial_exit_date': df_enhanced['time'].iloc[i],
                        'partial_exit_price': partial_exit_price,
                        'partial_exit_size': exit_size,
                        'partial_pnl': partial_pnl
                    })

            # Trailing stop implementatie voor het resterende deel (na partiële exit)
            if partial_exit_done and position > 0:
                # Bereken nieuwe trailing stop (2 ATR onder huidige prijs)
                potential_new_stop = df_enhanced['close'].iloc[i] - (atr_multiplier * atr_value)

                # Verhoog stop loss alleen als nieuwe stop hoger is dan bestaande
                if potential_new_stop > stop_loss:
                    stop_loss = potential_new_stop
                    df_enhanced.at[df_enhanced.index[i], 'stop_loss'] = stop_loss

        # Entry signaal
        if position == 0 and df_enhanced['entry_signal'].iloc[i]:
            entry_price = df_enhanced['close'].iloc[i]
            stop_loss = entry_price - (atr_multiplier * df_enhanced['atr'].iloc[i])

            # VERBETERING 2: Dynamische positiegrootte op basis van trendsterkte
            trend_strength = df_enhanced['trend_strength'].iloc[i]

            # Basis risico
            risk_amount = account_balance * risk_per_trade

            # Verhoog risico tot max 50% extra bij sterke trends
            if trend_strength > 0.01:  # Sterke trend
                risk_multiplier = min(1.5, 1 + trend_strength * 10)  # Max 50% extra
                risk_amount *= risk_multiplier

            # Bereken positiegrootte
            pips_at_risk = calculate_pips_at_risk(symbol, entry_price, stop_loss)
            pip_value = risk_amount / pips_at_risk if pips_at_risk > 0 else 0
            position_size = round(pip_value / calculate_pip_value(symbol, 1), 2) if pip_value > 0 else 0

            # Begrens positiegrootte
            position_size = max(0.01, min(position_size, 10.0))
            original_position_size = position_size

            # Entry uitvoeren
            position = 1
            partial_exit_done = False
            df_enhanced.at[df_enhanced.index[i], 'position'] = position
            df_enhanced.at[df_enhanced.index[i], 'stop_loss'] = stop_loss
            df_enhanced.at[df_enhanced.index[i], 'position_size'] = position_size
            df_enhanced.at[df_enhanced.index[i], 'partial_exit'] = partial_exit_done

            # Registreer trade
            results['enhanced']['trades'].append({
                'entry_date': df_enhanced['time'].iloc[i],
                'entry_price': entry_price,
                'position_size': position_size,
                'stop_loss': stop_loss,
                'risk_amount': risk_amount,
                'trend_strength': trend_strength
            })

        # Exit signaal of stop loss
        elif position > 0 and (df_enhanced['exit_signal'].iloc[i] or df_enhanced['low'].iloc[i] <= stop_loss):
            # Bereken P&L
            exit_price = df_enhanced['close'].iloc[i] if df_enhanced['exit_signal'].iloc[i] else stop_loss
            pnl = (exit_price - entry_price) * position_size * calculate_pip_value(symbol, 1)

            # Update account balance
            account_balance += pnl
            equity = account_balance

            # Registreer exit in laatste trade
            if results['enhanced']['trades']:
                last_trade = results['enhanced']['trades'][-1]
                full_pnl = pnl

                # Als er een partiële exit was, tel die winst mee
                if 'partial_pnl' in last_trade:
                    full_pnl += last_trade['partial_pnl']

                last_trade.update({
                    'exit_date': df_enhanced['time'].iloc[i],
                    'exit_price': exit_price,
                    'final_pnl': pnl,
                    'total_pnl': full_pnl,
                    'duration': (df_enhanced['time'].iloc[i] - last_trade['entry_date']).days
                })

            # Reset positie
            position = 0
            entry_price = 0
            stop_loss = 0
            position_size = 0
            partial_exit_done = False

            df_enhanced.at[df_enhanced.index[i], 'position'] = position
            df_enhanced.at[df_enhanced.index[i], 'equity'] = equity
            df_enhanced.at[df_enhanced.index[i], 'partial_exit'] = partial_exit_done

    # Sla eindequity op
    results['enhanced']['equity'] = equity

    # --- STAP 5: PRESTATIE VERGELIJKING ---

    # Bereken prestatiemetrieken voor beide strategieën
    metrics = {}

    for strategy in ['original', 'enhanced']:
        trades = results[strategy]['trades']

        # Skip als er geen trades zijn
        if not trades:
            metrics[strategy] = {
                'total_trades': 0,
                'win_rate': 0,
                'profit_factor': 0,
                'avg_win': 0,
                'avg_loss': 0,
                'max_profit': 0,
                'max_loss': 0,
                'total_profit': 0,
                'avg_duration': 0,
                'return': 0
            }
            continue

        # Filter voltooide trades
        completed_trades = [t for t in trades if 'exit_date' in t]

        if not completed_trades:
            continue

        # Bereken winst/verlies voor elke trade
        for trade in completed_trades:
            if strategy == 'original' or 'total_pnl' not in trade:
                trade['pnl'] = trade.get('pnl', 0)
            else:
                trade['pnl'] = trade.get('total_pnl', 0)

        # Winnende trades
        winning_trades = [t for t in completed_trades if t['pnl'] > 0]
        losing_trades = [t for t in completed_trades if t['pnl'] <= 0]

        # Basis statistieken
        total_trades = len(completed_trades)
        win_rate = len(winning_trades) / total_trades if total_trades > 0 else 0

        # Winst/verlies statistieken
        total_profit = sum(t['pnl'] for t in completed_trades)
        gross_profit = sum(t['pnl'] for t in winning_trades) if winning_trades else 0
        gross_loss = sum(abs(t['pnl']) for t in losing_trades) if losing_trades else 0

        profit_factor = gross_profit / gross_loss if gross_loss > 0 else float('inf')

        avg_win = gross_profit / len(winning_trades) if winning_trades else 0
        avg_loss = gross_loss / len(losing_trades) if losing_trades else 0

        max_profit = max(t['pnl'] for t in completed_trades) if completed_trades else 0
        max_loss = min(t['pnl'] for t in completed_trades) if completed_trades else 0

        # Duur statistieken
        avg_duration = sum(t['duration'] for t in completed_trades) / total_trades if total_trades > 0 else 0

        # Return berekening
        initial_balance = account_balance  # Uit de configuratie
        final_balance = results[strategy]['equity']
        percent_return = (final_balance / initial_balance - 1) * 100

        # Sla metrics op
        metrics[strategy] = {
            'total_trades': total_trades,
            'win_rate': win_rate,
            'profit_factor': profit_factor,
            'avg_win': avg_win,
            'avg_loss': avg_loss,
            'max_profit': max_profit,
            'max_loss': max_loss,
            'total_profit': total_profit,
            'avg_duration': avg_duration,
            'return': percent_return
        }

    # --- STAP 6: RESULTATEN TONEN ---

    # Toon samenvattende statistieken
    print("\nPrestatie Vergelijking:")
    print("-" * 60)
    print(f"{'Metriek':<20} {'Originele Strategie':<20} {'Verbeterde Strategie':<20}")
    print("-" * 60)

    # Lijst van te tonen metrieken
    display_metrics = [
        ('total_trades', 'Aantal Trades', '{:.0f}'),
        ('win_rate', 'Winratio', '{:.2%}'),
        ('profit_factor', 'Profit Factor', '{:.2f}'),
        ('avg_win', 'Gem. Winst', '${:.2f}'),
        ('avg_loss', 'Gem. Verlies', '${:.2f}'),
        ('max_profit', 'Max Winst', '${:.2f}'),
        ('max_loss', 'Max Verlies', '${:.2f}'),
        ('total_profit', 'Totale Winst', '${:.2f}'),
        ('avg_duration', 'Gem. Duur (dagen)', '{:.1f}'),
        ('return', 'Return', '{:.2f}%')
    ]

    for key, name, fmt in display_metrics:
        orig_val = metrics['original'][key] if 'original' in metrics else 0
        enh_val = metrics['enhanced'][key] if 'enhanced' in metrics else 0

        # Format de waarden
        if key == 'return':
            orig_str = fmt.format(orig_val)
            enh_str = fmt.format(enh_val)
        else:
            orig_str = fmt.format(orig_val)
            enh_str = fmt.format(enh_val)

        print(f"{name:<20} {orig_str:<20} {enh_str:<20}")

    # --- STAP 7: VISUALISATIE ---

    # Plot equity curves
    plt.figure(figsize=(15, 10))

    # Equity curves vergelijken
    plt.subplot(2, 1, 1)
    plt.plot(df_original['time'], df_original['equity'], label='Originele Strategie', color='blue')
    plt.plot(df_enhanced['time'], df_enhanced['equity'], label='Verbeterde Strategie', color='green')
    plt.title('Equity Curve Vergelijking')
    plt.ylabel('Account Equity ($)')
    plt.legend()
    plt.grid(True, alpha=0.3)

    # Plot posities
    plt.subplot(2, 1, 2)

    # Plot originele posities (blauwe invulling)
    plt.fill_between(df_original['time'], df_original['position'], color='blue', alpha=0.2, step='post', label='Originele Posities')

    # Plot verbeterde posities (groene invulling)
    plt.fill_between(df_enhanced['time'], df_enhanced['position'] * 0.8, color='green', alpha=0.2, step='post', label='Verbeterde Posities')

    # Markeer partiële exits
    partial_exits = df_enhanced[df_enhanced['partial_exit']]
    if not partial_exits.empty:
        plt.scatter(partial_exits['time'], partial_exits['position'] * 0.5, marker='o', color='red', label='Partiële Exits')

    plt.title('Positie Vergelijking')
    plt.ylabel('Positie')
    plt.xlabel('Datum')
    plt.yticks([0, 1])
    plt.legend()
    plt.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    return df_original, df_enhanced, metrics

# Voer de verbeterde backtest uit als we data hebben
if 'symbol_data' in locals() and symbol_data:
    # XAUUSD is onze focus vanwege de goede resultaten uit eerdere analyses
    focus_symbol = 'XAUUSD' if 'XAUUSD' in symbol_data else list(symbol_data.keys())[0]

    # Voer backtesting uit met verbeterde strategie
    original_df, enhanced_df, performance_metrics = backtest_enhanced_strategy(
        symbol_data[focus_symbol],
        focus_symbol,
        account_balance=100000,
        risk_per_trade=0.01
    )


VERBETERDE BACKTESTING: ORIGINEEL VS. VERBETERD


Prestatie Vergelijking:
------------------------------------------------------------
Metriek              Originele Strategie  Verbeterde Strategie
------------------------------------------------------------
Aantal Trades        70                   35                  
Winratio             35.71%               54.29%              
Profit Factor        1.41                 0.68                
Gem. Winst           $219.33              $70.50              
Gem. Verlies         $86.57               $123.94             
Max Winst            $807.57              $165.41             
Max Verlies          $-187.28             $-210.21            
Totale Winst         $1587.38             $-643.56            
Gem. Duur (dagen)    5.0                  2.0                 
Return               2.25%                0.00%               
